In [55]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [56]:
application_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [57]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"],inplace=True)

In [58]:
# Determine the number of unique values in each column.
all_col = application_df.nunique()
more_than_ten = all_col[all_col.values > 10].index
more_than_ten

Index(['APPLICATION_TYPE', 'CLASSIFICATION', 'ASK_AMT'], dtype='object')

In [59]:
# optimization method 1: changing the binning for both 'APPLICATION_TYPE'
# and 'CLASSIFICATION' to 200

In [60]:
def bin_col(df,col_name,limit):
    count = df[col_name].value_counts()
    replace = list(count[count < limit].index)
    for app in replace:
        df[col_name] = df[col_name].replace(app,"Other")
    return df

In [61]:
application_df = bin_col(application_df,'APPLICATION_TYPE',200)
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [62]:
application_df = bin_col(application_df,'CLASSIFICATION',200)
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [63]:
# Generate our categorical variable lists
cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
# Create a OneHotEncoder instanc
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(cat)
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(cat,1)

C:\Users\afrap\anaconda3\envs\mlenv\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\afrap\AppData\Local\Temp\ipykernel_16084\3169484359.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(cat,1)


In [64]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

In [65]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [66]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


In [67]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [68]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 718us/step - loss: 0.5749 - accuracy: 0.7179
Epoch 2/100
804/804 [==============================] - 1s 712us/step - loss: 0.5574 - accuracy: 0.7266
Epoch 3/100
804/804 [==============================] - 1s 710us/step - loss: 0.5556 - accuracy: 0.7249
Epoch 4/100
804/804 [==============================] - 1s 709us/step - loss: 0.5525 - accuracy: 0.7288
Epoch 5/100
804/804 [==============================] - 1s 713us/step - loss: 0.5520 - accuracy: 0.7278
Epoch 6/100
804/804 [==============================] - 1s 711us/step - loss: 0.5509 - accuracy: 0.7291
Epoch 7/100
804/804 [==============================] - 1s 711us/step - loss: 0.5497 - accuracy: 0.7302
Epoch 8/100
804/804 [==============================] - 1s 712us/step - loss: 0.5492 - accuracy: 0.7303
Epoch 9/100
804/804 [==============================] - 1s 707us/step - loss: 0.5489 - accuracy: 0.7304
Epoch 10/100
804/804 [==============================] - 1s 716us/step - l

804/804 [==============================] - 1s 725us/step - loss: 0.5377 - accuracy: 0.7369
Epoch 80/100
804/804 [==============================] - 1s 728us/step - loss: 0.5369 - accuracy: 0.7380
Epoch 81/100
804/804 [==============================] - 1s 735us/step - loss: 0.5372 - accuracy: 0.7374
Epoch 82/100
804/804 [==============================] - 1s 728us/step - loss: 0.5375 - accuracy: 0.7373
Epoch 83/100
804/804 [==============================] - 1s 729us/step - loss: 0.5368 - accuracy: 0.7379
Epoch 84/100
804/804 [==============================] - 1s 726us/step - loss: 0.5369 - accuracy: 0.7378
Epoch 85/100
804/804 [==============================] - 1s 743us/step - loss: 0.5369 - accuracy: 0.7377
Epoch 86/100
804/804 [==============================] - 1s 734us/step - loss: 0.5375 - accuracy: 0.7375
Epoch 87/100
804/804 [==============================] - 1s 754us/step - loss: 0.5370 - accuracy: 0.7372
Epoch 88/100
804/804 [==============================] - 1s 729us/step - loss:

In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5471 - accuracy: 0.7399 - 214ms/epoch - 800us/step
Loss: 0.5470985174179077, Accuracy: 0.7399417161941528


In [49]:
# optimization method 2: Using 200 for binning and increasing the number of neurons in the hidden layers

In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 60

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_new = nn_new.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 757us/step - loss: 0.5741 - accuracy: 0.7176
Epoch 2/100
804/804 [==============================] - 1s 756us/step - loss: 0.5574 - accuracy: 0.7263
Epoch 3/100
804/804 [==============================] - 1s 764us/step - loss: 0.5546 - accuracy: 0.7274
Epoch 4/100
804/804 [==============================] - 1s 801us/step - loss: 0.5525 - accuracy: 0.7281
Epoch 5/100
804/804 [==============================] - 1s 781us/step - loss: 0.5510 - accuracy: 0.7278
Epoch 6/100
804/804 [==============================] - 1s 763us/step - loss: 0.5505 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 1s 749us/step - loss: 0.5501 - accuracy: 0.7288
Epoch 8/100
804/804 [==============================] - 1s 759us/step - loss: 0.5489 - accuracy: 0.7306
Epoch 9/100
804/804 [==============================] - 1s 754us/step - loss: 0.5485 - accuracy: 0.7321
Epoch 10/100
804/804 [==============================] - 1s 778us/step - l

In [69]:
# optimization method 3: increasing number of hidden layers

In [70]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 60
hidden_nodes_layer4 = 60

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# forth hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))


# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_new = nn_new.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 837us/step - loss: 0.5715 - accuracy: 0.7190
Epoch 2/100
804/804 [==============================] - 1s 833us/step - loss: 0.5567 - accuracy: 0.7269
Epoch 3/100
804/804 [==============================] - 1s 855us/step - loss: 0.5540 - accuracy: 0.7272
Epoch 4/100
804/804 [==============================] - 1s 859us/step - loss: 0.5527 - accuracy: 0.7265
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7299
Epoch 6/100
804/804 [==============================] - 1s 942us/step - loss: 0.5503 - accuracy: 0.7301
Epoch 7/100
804/804 [==============================] - 1s 991us/step - loss: 0.5494 - accuracy: 0.7308
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7318
Epoch 9/100
804/804 [==============================] - 1s 856us/step - loss: 0.5480 - accuracy: 0.7318
Epoch 10/100
804/804 [==============================] - 1s 871us/step - loss:

804/804 [==============================] - 1s 850us/step - loss: 0.5345 - accuracy: 0.7386
Epoch 80/100
804/804 [==============================] - 1s 870us/step - loss: 0.5343 - accuracy: 0.7383
Epoch 81/100
804/804 [==============================] - 1s 860us/step - loss: 0.5346 - accuracy: 0.7383
Epoch 82/100
804/804 [==============================] - 1s 852us/step - loss: 0.5349 - accuracy: 0.7376
Epoch 83/100
804/804 [==============================] - 1s 874us/step - loss: 0.5344 - accuracy: 0.7390
Epoch 84/100
804/804 [==============================] - 1s 845us/step - loss: 0.5348 - accuracy: 0.7379
Epoch 85/100
804/804 [==============================] - 1s 869us/step - loss: 0.5349 - accuracy: 0.7378
Epoch 86/100
804/804 [==============================] - 1s 853us/step - loss: 0.5347 - accuracy: 0.7368
Epoch 87/100
804/804 [==============================] - 1s 848us/step - loss: 0.5343 - accuracy: 0.7383
Epoch 88/100
804/804 [==============================] - 1s 865us/step - loss:

In [ ]:
# optimization method 4:Increasing number of epochs

In [73]:
# Train the model
fit_model_new = nn_new.fit(X_train_scaled,y_train,epochs=500)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/500
804/804 [==============================] - 1s 855us/step - loss: 0.5320 - accuracy: 0.7389
Epoch 2/500
804/804 [==============================] - 1s 870us/step - loss: 0.5324 - accuracy: 0.7391
Epoch 3/500
804/804 [==============================] - 1s 856us/step - loss: 0.5329 - accuracy: 0.7379
Epoch 4/500
804/804 [==============================] - 1s 863us/step - loss: 0.5332 - accuracy: 0.7394
Epoch 5/500
804/804 [==============================] - 1s 862us/step - loss: 0.5322 - accuracy: 0.7390
Epoch 6/500
804/804 [==============================] - 1s 859us/step - loss: 0.5365 - accuracy: 0.7380
Epoch 7/500
804/804 [==============================] - 1s 893us/step - loss: 0.5319 - accuracy: 0.7384
Epoch 8/500
804/804 [==============================] - 1s 865us/step - loss: 0.5318 - accuracy: 0.7390
Epoch 9/500
804/804 [==============================] - 1s 893us/step - loss: 0.5374 - accuracy: 0.7385
Epoch 10/500
804/804 [==============================] - 1s 898us/step - l

804/804 [==============================] - 1s 864us/step - loss: 0.5315 - accuracy: 0.7399
Epoch 80/500
804/804 [==============================] - 1s 873us/step - loss: 0.5341 - accuracy: 0.7390
Epoch 81/500
804/804 [==============================] - 1s 881us/step - loss: 0.5316 - accuracy: 0.7385
Epoch 82/500
804/804 [==============================] - 1s 860us/step - loss: 0.5311 - accuracy: 0.7397
Epoch 83/500
804/804 [==============================] - 1s 861us/step - loss: 0.5318 - accuracy: 0.7397
Epoch 84/500
804/804 [==============================] - 1s 866us/step - loss: 0.5335 - accuracy: 0.7390
Epoch 85/500
804/804 [==============================] - 1s 862us/step - loss: 0.5319 - accuracy: 0.7389
Epoch 86/500
804/804 [==============================] - 1s 861us/step - loss: 0.5313 - accuracy: 0.7393
Epoch 87/500
804/804 [==============================] - 1s 868us/step - loss: 0.5317 - accuracy: 0.7400
Epoch 88/500
804/804 [==============================] - 1s 860us/step - loss:

804/804 [==============================] - 1s 866us/step - loss: 0.5306 - accuracy: 0.7395
Epoch 158/500
804/804 [==============================] - 1s 865us/step - loss: 0.5315 - accuracy: 0.7397
Epoch 159/500
804/804 [==============================] - 1s 861us/step - loss: 0.5305 - accuracy: 0.7402
Epoch 160/500
804/804 [==============================] - 1s 870us/step - loss: 0.5311 - accuracy: 0.7396
Epoch 161/500
804/804 [==============================] - 1s 873us/step - loss: 0.5305 - accuracy: 0.7398
Epoch 162/500
804/804 [==============================] - 1s 894us/step - loss: 0.5310 - accuracy: 0.7393
Epoch 163/500
804/804 [==============================] - 1s 861us/step - loss: 0.5307 - accuracy: 0.7397
Epoch 164/500
804/804 [==============================] - 1s 865us/step - loss: 0.5344 - accuracy: 0.7384
Epoch 165/500
804/804 [==============================] - 1s 864us/step - loss: 0.5316 - accuracy: 0.7397
Epoch 166/500
804/804 [==============================] - 1s 864us/ste

804/804 [==============================] - 1s 868us/step - loss: 0.5305 - accuracy: 0.7399
Epoch 236/500
804/804 [==============================] - 1s 865us/step - loss: 0.5301 - accuracy: 0.7404
Epoch 237/500
804/804 [==============================] - 1s 863us/step - loss: 0.5305 - accuracy: 0.7400
Epoch 238/500
804/804 [==============================] - 1s 865us/step - loss: 0.5304 - accuracy: 0.7395
Epoch 239/500
804/804 [==============================] - 1s 865us/step - loss: 0.5311 - accuracy: 0.7390
Epoch 240/500
804/804 [==============================] - 1s 860us/step - loss: 0.5353 - accuracy: 0.7394
Epoch 241/500
804/804 [==============================] - 1s 868us/step - loss: 0.5305 - accuracy: 0.7399
Epoch 242/500
804/804 [==============================] - 1s 858us/step - loss: 0.5298 - accuracy: 0.7399
Epoch 243/500
804/804 [==============================] - 1s 861us/step - loss: 0.5299 - accuracy: 0.7393
Epoch 244/500
804/804 [==============================] - 1s 865us/ste

804/804 [==============================] - 1s 866us/step - loss: 0.5304 - accuracy: 0.7399
Epoch 314/500
804/804 [==============================] - 1s 859us/step - loss: 0.5314 - accuracy: 0.7402
Epoch 315/500
804/804 [==============================] - 1s 868us/step - loss: 0.5303 - accuracy: 0.7401
Epoch 316/500
804/804 [==============================] - 1s 860us/step - loss: 0.5331 - accuracy: 0.7404
Epoch 317/500
804/804 [==============================] - 1s 857us/step - loss: 0.5302 - accuracy: 0.7402
Epoch 318/500
804/804 [==============================] - 1s 865us/step - loss: 0.5305 - accuracy: 0.7395
Epoch 319/500
804/804 [==============================] - 1s 860us/step - loss: 0.5309 - accuracy: 0.7387
Epoch 320/500
804/804 [==============================] - 1s 861us/step - loss: 0.5303 - accuracy: 0.7392
Epoch 321/500
804/804 [==============================] - 1s 866us/step - loss: 0.5304 - accuracy: 0.7382
Epoch 322/500
804/804 [==============================] - 1s 874us/ste

804/804 [==============================] - 1s 862us/step - loss: 0.5302 - accuracy: 0.7392
Epoch 392/500
804/804 [==============================] - 1s 858us/step - loss: 0.5309 - accuracy: 0.7399
Epoch 393/500
804/804 [==============================] - 1s 863us/step - loss: 0.5302 - accuracy: 0.7402
Epoch 394/500
804/804 [==============================] - 1s 858us/step - loss: 0.5298 - accuracy: 0.7396
Epoch 395/500
804/804 [==============================] - 1s 866us/step - loss: 0.5297 - accuracy: 0.7401
Epoch 396/500
804/804 [==============================] - 1s 870us/step - loss: 0.5297 - accuracy: 0.7400
Epoch 397/500
804/804 [==============================] - 1s 860us/step - loss: 0.5300 - accuracy: 0.7404
Epoch 398/500
804/804 [==============================] - 1s 861us/step - loss: 0.5337 - accuracy: 0.7385
Epoch 399/500
804/804 [==============================] - 1s 863us/step - loss: 0.5305 - accuracy: 0.7393
Epoch 400/500
804/804 [==============================] - 1s 864us/ste

804/804 [==============================] - 1s 873us/step - loss: 0.5311 - accuracy: 0.7406
Epoch 470/500
804/804 [==============================] - 1s 883us/step - loss: 0.5298 - accuracy: 0.7402
Epoch 471/500
804/804 [==============================] - 1s 874us/step - loss: 0.5295 - accuracy: 0.7400
Epoch 472/500
804/804 [==============================] - 1s 884us/step - loss: 0.5292 - accuracy: 0.7408
Epoch 473/500
804/804 [==============================] - 1s 875us/step - loss: 0.5292 - accuracy: 0.7394
Epoch 474/500
804/804 [==============================] - 1s 881us/step - loss: 0.5291 - accuracy: 0.7411
Epoch 475/500
804/804 [==============================] - 1s 879us/step - loss: 0.5296 - accuracy: 0.7404
Epoch 476/500
804/804 [==============================] - 1s 875us/step - loss: 0.5326 - accuracy: 0.7403
Epoch 477/500
804/804 [==============================] - 1s 880us/step - loss: 0.5294 - accuracy: 0.7407
Epoch 478/500
804/804 [==============================] - 1s 884us/ste